In [ ]:
# Constants
MODEL_NAME = "facebook/opt-350m"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLUSTERS = 16
MAX_SAMPLES = 1000
SENSITIVE_LAYER_PERCENTAGE = 0.3
EPOCHS = 3
BATCH_SIZE = 16
LEARNING_RATE = 1e-4
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Preprocessing Data for HellaSwag
def preprocess_data(dataset, dataset_name, max_samples=100, max_length=512):
    inputs = []
    labels = []

    for i, example in enumerate(dataset):
        if i >= max_samples:
            break

        if dataset_name == "hellaswag":
            # Specific preprocessing for HellaSwag
            context = example['ctx']
            ending = example['endings'][example['label']]  # Selecting the correct ending
            text = f"Context: {context} Ending: {ending}"
            label = example['label']
        elif dataset_name == "piqa":
            # Specific preprocessing for PIQA
            text = f"Question: {example['goal']} Choice: {example['sol1']}"
            label = example['label']
        elif dataset_name == "boolq":
            # Specific preprocessing for BoolQ
            text = f"Question: {example['question']} Context: {example.get('context', example.get('passage', '')}"
            label = int(example['answer'])  # Convert boolean to integer
        else:
            raise ValueError("Unsupported dataset format or missing keys.")

        if label is None:
            print(f"Skipping example due to missing label: {example}")
            continue  # Skip this example

        inputs.append(text)
        labels.append(label)

    return inputs, labels

# Main Workflow for HellaSwag
def main(dataset, dataset_name):
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

    inputs, labels = preprocess_data(dataset, dataset_name, MAX_SAMPLES)

    if not inputs or not labels:
        raise ValueError("The dataset is empty or preprocessing returned no data.")

    tokenized_inputs = tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")
    dataloader = create_dataloader(tokenized_inputs, labels, BATCH_SIZE)

    sensitivities = calculate_layer_sensitivities(model)
    sensitive_layers = get_top_sensitive_layers(sensitivities, SENSITIVE_LAYER_PERCENTAGE)

    print(f"Sensitive Layers for {dataset_name}:", sensitive_layers)

    cluster_layers(model, NUM_CLUSTERS)

    start_time = time.time()
    accuracy_before = evaluate_model(model, dataloader)

    fine_tune_model(model, dataloader, sensitive_layers)

    accuracy_after = evaluate_model(model, dataloader)
    end_time = time.time()
    print(f"Accuracy Before Fine-Tuning on {dataset_name}: {accuracy_before}")
    print(f"Accuracy After Fine-Tuning on {dataset_name}: {accuracy_after}")
    print(f"Accuracy Drop on {dataset_name}: {accuracy_before - accuracy_after}")
    print(f"Total Time for {dataset_name}: {end_time - start_time} seconds")

# Example Dataset for HellaSwag
from datasets import load_dataset

dataset_name = "hellaswag"
dataset = load_dataset(dataset_name, split="train[:1000]")
main(dataset, dataset_name)
